**Input**:
    - pred_points: predicted list of 3d coordinates that are points along the neuron curve
    - ann_points: annotated list of 3d coordinates that are points along the neuron curve
        - of the form: (x, y, z, i) where i is the index of that point along the curve, with -1 indicating start of a curve
        
**Pseudocode**:
1. Iterate through the annotated neural tracts and calculate the lengths of each of the tracts.
2. Sum the all the lengths of the annotated neural tracts for that subvolume.
3. Repeat steps 1 and 2 for predicted neural tracts.
4. Take the difference between the sums of the lengths of the predicted and annotated neural tracts.
        

# Getting sum of intensities

In [1]:
%matplotlib inline

import sys
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt

from math import sqrt, hypot, log, pi

import itertools as itt

from skimage.filters import threshold_otsu, threshold_local

In [5]:
# Loading the stack
stack_0 = sitk.ReadImage('../../sample_data/tractography/tractography_0.tif')
stack_0 = sitk.GetArrayFromImage(stack_0)

# Normalizing
image_0 = stack_0 * np.float32(255.0 / stack_0.max())
# img_0 = stack_0 * np.uint16(255 / stack_0.max())
print('shape:', image_0.shape)
print('max:', image_0.max())
print('min:', image_0.min())

('shape:', (100, 500, 500))
('max:', 255.0)
('min:', 0.10505836)


In [14]:
(z_dim, y_dim, x_dim) = image_0.shape

In [16]:
# Thresholding: making all voxels with intensity less than half of otsu's 0.
otsus = []

for k in range(0, z_dim):
    s = image_0[k,:,:]
    otsus.append(threshold_otsu(s))
    
stack_otsu_thresh = np.median(otsus)
print(np.median(otsus))

3.27885612116


In [17]:
image_0_thresh = image_0 > stack_otsu_thresh

In [19]:
image_0_sum = np.sum(image_0_thresh)
print(image_0_sum)

683794


# Getting sums of lengths of annotations

In [38]:
import pandas as pd

In [61]:
ann = pd.read_csv('../../annotation_csv/tractography/tractography_0.swc', header=None, delim_whitespace=True, skiprows=2)
print(ann)

       0  1        2        3       4    5    6
0      1  3  164.847  156.714  36.008  1.0   -1
1      2  3  160.028  159.115  35.887  1.0    1
2      3  3  155.373  164.337  35.655  1.0    2
3      4  3  151.526  168.837  36.458  1.0    3
4      5  3  148.125  173.987  36.246  1.0    4
5      6  3  144.447  177.196  35.795  1.0    5
6      7  3  139.054  180.932  35.426  1.0    6
7      8  3  133.591  185.292  35.050  1.0    7
8      9  3  131.250  187.161  34.888  1.0    8
9     10  3  129.065  189.048  34.730  1.0    9
10    11  3  127.972  191.057  34.599  1.0   10
11    12  3  127.973  193.189  34.494  1.0   11
12    13  3  127.817  194.592  34.420  1.0   12
13    14  3  125.476  199.304  34.118  1.0   13
14    15  3  121.575  203.840  33.779  1.0   14
15    16  3  116.779  207.690  34.337  1.0   15
16    17  3  111.400  211.987  34.077  1.0   16
17    18  3  106.110  214.788  33.783  1.0   17
18    19  3  100.959  216.342  33.554  1.0   18
19    20  3   95.201  218.679  33.188  1

In [62]:
# ann_points = ann.ix[:,2:4]
ann_points = ann[[2,3,4,6]]
print(ann_points)

           2        3       4    6
0    164.847  156.714  36.008   -1
1    160.028  159.115  35.887    1
2    155.373  164.337  35.655    2
3    151.526  168.837  36.458    3
4    148.125  173.987  36.246    4
5    144.447  177.196  35.795    5
6    139.054  180.932  35.426    6
7    133.591  185.292  35.050    7
8    131.250  187.161  34.888    8
9    129.065  189.048  34.730    9
10   127.972  191.057  34.599   10
11   127.973  193.189  34.494   11
12   127.817  194.592  34.420   12
13   125.476  199.304  34.118   13
14   121.575  203.840  33.779   14
15   116.779  207.690  34.337   15
16   111.400  211.987  34.077   16
17   106.110  214.788  33.783   17
18   100.959  216.342  33.554   18
19    95.201  218.679  33.188   19
20    88.668  224.000  32.097   20
21    83.782  229.242  31.893   21
22    80.389  235.326  31.891   22
23    79.748  237.316  31.981   23
24    79.186  238.653  33.150   24
25    78.373  240.624  33.230   25
26    76.434  242.883  32.854   26
27    75.142  244.38

In [63]:
ann_points.columns = ['x', 'y', 'z', 'i']
print(ann_points)

           x        y       z    i
0    164.847  156.714  36.008   -1
1    160.028  159.115  35.887    1
2    155.373  164.337  35.655    2
3    151.526  168.837  36.458    3
4    148.125  173.987  36.246    4
5    144.447  177.196  35.795    5
6    139.054  180.932  35.426    6
7    133.591  185.292  35.050    7
8    131.250  187.161  34.888    8
9    129.065  189.048  34.730    9
10   127.972  191.057  34.599   10
11   127.973  193.189  34.494   11
12   127.817  194.592  34.420   12
13   125.476  199.304  34.118   13
14   121.575  203.840  33.779   14
15   116.779  207.690  34.337   15
16   111.400  211.987  34.077   16
17   106.110  214.788  33.783   17
18   100.959  216.342  33.554   18
19    95.201  218.679  33.188   19
20    88.668  224.000  32.097   20
21    83.782  229.242  31.893   21
22    80.389  235.326  31.891   22
23    79.748  237.316  31.981   23
24    79.186  238.653  33.150   24
25    78.373  240.624  33.230   25
26    76.434  242.883  32.854   26
27    75.142  244.38

In [65]:
def euclidian_dist(p1, p2):
    return np.linalg.norm(p1 - p2)

In [90]:
def get_total_length(ann_df):
    ann_total_length = 0
    
    current_curve_distance = 0
    prev_row = None;
    for index, row in ann_df.iterrows():
        if index == 0:
            prev_row = row
            continue
            
        if row['i'] == -1:
            # if the next point is part of a different curve
            ann_total_length += current_curve_distance
#             print('ann')
#             print(ann_total_length)
            current_curve_distance = 0
            prev_row = row
            continue
            
        p1 = np.array([row['x'], row['y'], row['z']])
        p2 = np.array([prev_row['x'], prev_row['y'], prev_row['z']])
#         print(euclidian_dist(p1, p2))
        current_curve_distance += euclidian_dist(p1, p2)
        prev_row = row
    
    ann_total_length += current_curve_distance
    
    return ann_total_length
        
        
#     current_curve_distance = 0
#     for i in xrange(1, len(ann_points)):
#         if ann_points[i, 3] == -1:
#             # if the next point is part of a different curve
#             ann_total_dist += current_curve_distance
#             current_curve_distance = 0
#         current_curve_distance += euclidian_distance(ann_points[i-1, 0:4], ann_points[i, 0:4])
        
#     current_curve_distance = 0
#     for i in xrange(1, len(pred_points)):
#         if pred_points[i, 3] == -1:
#             # if the next point is part of a different curve
#             pred_total_dist += current_curve_distance
#             current_curve_distance = 0
#         current_curve_distance += euclidian_distance(pred_points[i-1, 0:4], pred_points[i, 0:4])
    
#     return (pred_total_dist - ann_total_dist)

In [92]:
ann_total_length = get_total_length(ann_points)
print(ann_total_length)

2527.52506504


In [76]:
test = pd.read_csv('../../annotation_csv/tractography/test.swc', header=None, delim_whitespace=True, skiprows=2)
print(test)


   0  1  2  3  4    5  6
0  1  3  0  0  0  1.0 -1
1  2  3  1  0  0  1.0  1
2  3  3  1  1  0  1.0  2
3  4  3  0  0  0  1.0 -1
4  5  3  0  0  1  1.0  4
5  6  3  0  1  1  1.0  5
6  7  3  1  1  1  1.0  6


In [78]:
test_points = test[[2,3,4,6]]
test_points.columns = ['x', 'y', 'z', 'i']
print(test_points)

   x  y  z  i
0  0  0  0 -1
1  1  0  0  1
2  1  1  0  2
3  0  0  0 -1
4  0  0  1  4
5  0  1  1  5
6  1  1  1  6


In [91]:
test_total_length = get_total_length(test_points)

print('total length')
print(test_total_length)

total length
5.0
